# Airfare and Traffic Data Analysis

In [2]:
# Import required libraries
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

## Importing Dataset

** Table 6 is the table from USDoT.
** The dropped columns were keys or other info useful when working with other tables, but not in this project.
** The two city columns create a city-pair, which is used to provide info on the number of passengers and fares between the pair.

In [3]:
drop_cols = ['tbl', 'citymarketid_1', 'citymarketid_2', 'table_1_flag', 'tbl6pk', 'Geocoded_City1', 'Geocoded_City2']
df = pd.read_csv('..\Consumer_Airfare_Report__Table_6_-_Contiguous_State_City-Pair_Markets_That_Average_At_Least_10_Passengers_Per_Day.csv',
                 usecols=(lambda x: x not in drop_cols)
                 )
carriers_df = pd.read_csv('..\\carrier_codes.csv') # The airline carrier codes list from DoT

df.head()

,Year,quarter,city1,city2,nsmiles,passengers,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low
0,2004,2,"Oklahoma City, OK","West Palm Beach/Palm Beach, FL",1197,11,242.51,DL,0.51,235.98,DL,0.51,235.98
1,2003,2,"Nashville, TN","West Palm Beach/Palm Beach, FL",758,138,136.00,WN,0.73,126.54,WN,0.73,126.54
2,2004,4,"Lincoln, NE","San Diego, CA",1259,19,193.88,UA,0.84,194.65,NW,0.13,176.53
3,2004,2,"Kansas City, MO","Wausau/Mosinee/Stevens Point, WI",459,13,199.57,NW,0.70,201.34,UA,0.13,195.58
4,2003,2,"Allentown/Bethlehem/Easton, PA","Denver, CO",1539,29,272.83,UA,0.32,321.47,DL,0.17,201.91


In [4]:
df.rename(columns=str.lower, inplace=True)
df.rename(columns={
    'nsmiles': 'nonstop_miles',
    'carrier_lg': 'carrier_largest',
    'large_ms': 'market_share_largest',
    'fare_lg': 'fare_largest',
    'carrier_low': 'carrier_lowfare',
    'lf_ms': 'market_share_lowfare',
    'fare_low': 'fare_lowfare'
    },
    inplace=True)

df['city-pair'] = df['city1'] + " - " + df['city2']
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581832 entries, 0 to 581831
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   year                  581832 non-null  int64  
 1   quarter               581832 non-null  int64  
 2   city1                 581832 non-null  object 
 3   city2                 581832 non-null  object 
 4   nonstop_miles         581832 non-null  int64  
 5   passengers            581832 non-null  int64  
 6   fare                  581832 non-null  float64
 7   carrier_largest       581690 non-null  object 
 8   market_share_largest  581690 non-null  float64
 9   fare_largest          581690 non-null  float64
 10  carrier_lowfare       581264 non-null  object 
 11  market_share_lowfare  581264 non-null  float64
 12  fare_lowfare          581264 non-null  float64
 13  city-pair             581832 non-null  object 
dtypes: float64(5), int64(4), object(5)
memory usage: 62.

,year,quarter,city1,city2,nonstop_miles,passengers,fare,carrier_largest,market_share_largest,fare_largest,carrier_lowfare,market_share_lowfare,fare_lowfare,city-pair
0,2004,2,"Oklahoma City, OK","West Palm Beach/Palm Beach, FL",1197,11,242.51,DL,0.51,235.98,DL,0.51,235.98,"Oklahoma City, OK - West Palm Beach/Palm Beach..."
1,2003,2,"Nashville, TN","West Palm Beach/Palm Beach, FL",758,138,136.00,WN,0.73,126.54,WN,0.73,126.54,"Nashville, TN - West Palm Beach/Palm Beach, FL"
2,2004,4,"Lincoln, NE","San Diego, CA",1259,19,193.88,UA,0.84,194.65,NW,0.13,176.53,"Lincoln, NE - San Diego, CA"
3,2004,2,"Kansas City, MO","Wausau/Mosinee/Stevens Point, WI",459,13,199.57,NW,0.70,201.34,UA,0.13,195.58,"Kansas City, MO - Wausau/Mosinee/Stevens Point..."
4,2003,2,"Allentown/Bethlehem/Easton, PA","Denver, CO",1539,29,272.83,UA,0.32,321.47,DL,0.17,201.91,"Allentown/Bethlehem/Easton, PA - Denver, CO"


### Fixing missing values and nulls

In [5]:
missing_df = df[df['carrier_largest'].isna()]
missing_df.head()

,year,quarter,city1,city2,nonstop_miles,passengers,fare,carrier_largest,market_share_largest,fare_largest,carrier_lowfare,market_share_lowfare,fare_lowfare,city-pair
692,1997,1,"Port Angeles, WA","San Francisco, CA (Metropolitan Area)",747,11,105.00,NaN,NaN,NaN,NaN,NaN,NaN,"Port Angeles, WA - San Francisco, CA (Metropol..."
3076,1996,4,"Las Vegas, NV","Redding, CA",493,10,105.80,NaN,NaN,NaN,NaN,NaN,NaN,"Las Vegas, NV - Redding, CA"
3640,1996,1,"Los Angeles, CA (Metropolitan Area)","Santa Rosa, CA",433,44,107.03,NaN,NaN,NaN,NaN,NaN,NaN,"Los Angeles, CA (Metropolitan Area) - Santa Ro..."
4824,1997,1,"Dallas/Fort Worth, TX","Victoria, TX",279,12,93.50,NaN,NaN,NaN,NaN,NaN,NaN,"Dallas/Fort Worth, TX - Victoria, TX"
8040,1997,2,"Los Angeles, CA (Metropolitan Area)","Modesto, CA",323,19,114.33,NaN,NaN,NaN,NaN,NaN,NaN,"Los Angeles, CA (Metropolitan Area) - Modesto, CA"


In [6]:
missing_df.describe()

,year,quarter,nonstop_miles,passengers,fare,market_share_largest,fare_largest,market_share_lowfare,fare_lowfare
count,142.000000,142.000000,142.000000,142.000000,142.000000,0.0,0.0,0.0,0.0
mean,1997.204225,2.669014,777.246479,20.302817,160.510493,NaN,NaN,NaN,NaN
std,2.766411,1.121814,467.683641,15.725477,71.572006,NaN,NaN,NaN,NaN
min,1996.000000,1.000000,226.000000,10.000000,83.600000,NaN,NaN,NaN,NaN
25%,1996.000000,2.000000,476.000000,11.000000,105.882500,NaN,NaN,NaN,NaN
50%,1997.000000,3.000000,685.000000,14.000000,127.725000,NaN,NaN,NaN,NaN
75%,1997.000000,4.000000,954.000000,22.000000,199.112500,NaN,NaN,NaN,NaN
max,2015.000000,4.000000,2415.000000,98.000000,453.990000,NaN,NaN,NaN,NaN


There are a number of rows that don't have market share data at all. Those rows will be deleted. The rows that have a market share leader but no lowest fare data will be preserved by adding the largest carrier as the lowest fare carrier.  This is a small number of entries that will not adversely affect the analysis.

In [7]:
# 
df = df.dropna(subset=['carrier_largest'])
df = df.reset_index(drop=True)

df.carrier_lowfare.fillna(df.carrier_largest, inplace=True)
df.market_share_lowfare.fillna(df.market_share_largest, inplace=True)
df.fare_lowfare.fillna(df.fare_largest, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581690 entries, 0 to 581689
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   year                  581690 non-null  int64  
 1   quarter               581690 non-null  int64  
 2   city1                 581690 non-null  object 
 3   city2                 581690 non-null  object 
 4   nonstop_miles         581690 non-null  int64  
 5   passengers            581690 non-null  int64  
 6   fare                  581690 non-null  float64
 7   carrier_largest       581690 non-null  object 
 8   market_share_largest  581690 non-null  float64
 9   fare_largest          581690 non-null  float64
 10  carrier_lowfare       581690 non-null  object 
 11  market_share_lowfare  581690 non-null  float64
 12  fare_lowfare          581690 non-null  float64
 13  city-pair             581690 non-null  object 
dtypes: float64(5), int64(4), object(5)
memory usage: 62.

## Geocoding and Decoding Carrier Codes

The geocoding in the dataset was a mess. It was implemented only in certain years, and the format was not uniform. The following will export a cities list to query outside sources for geocoding.

In [8]:
# Query the df for a list of cities to be geocoded.
city_df = pd.DataFrame(np.unique(df[['city1','city2']].values),columns=['address'])
city_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   address  376 non-null    object
dtypes: object(1)
memory usage: 3.1+ KB


In [9]:
city_df.to_csv('..\\cities.csv')

In [10]:
coords_df = pd.read_csv('..\\city_locations.csv')  # imported geocode mostly from geocod.io

In [11]:
coords_df.info()
coords_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   address  376 non-null    object 
 1   lat      376 non-null    float64
 2   lon      376 non-null    float64
dtypes: float64(2), object(1)
memory usage: 8.9+ KB


,address,lat,lon
0,"Aberdeen, SD",45.464981,-98.487813
1,"Abilene, TX",32.446450,-99.747591
2,"Albany, GA",31.578206,-84.155681
3,"Albany, NY",42.651167,-73.754968
4,"Albuquerque, NM",35.084103,-106.650985


In [12]:
# Two left joins to incorporate geocode to the two city columns
df = df.merge(coords_df, left_on='city1', right_on='address', how='left')
df.rename(columns={'lat': 'city1_lat', 'lon': 'city1_lon'}, inplace=True)
df.drop(['address'], axis=1, inplace=True)

df = df.merge(coords_df, left_on='city2', right_on='address', how='left')
df.rename(columns={'lat': 'city2_lat', 'lon': 'city2_lon'}, inplace=True)
df.drop(['address'], axis=1, inplace=True)

df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 581690 entries, 0 to 581689
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   year                  581690 non-null  int64  
 1   quarter               581690 non-null  int64  
 2   city1                 581690 non-null  object 
 3   city2                 581690 non-null  object 
 4   nonstop_miles         581690 non-null  int64  
 5   passengers            581690 non-null  int64  
 6   fare                  581690 non-null  float64
 7   carrier_largest       581690 non-null  object 
 8   market_share_largest  581690 non-null  float64
 9   fare_largest          581690 non-null  float64
 10  carrier_lowfare       581690 non-null  object 
 11  market_share_lowfare  581690 non-null  float64
 12  fare_lowfare          581690 non-null  float64
 13  city-pair             581690 non-null  object 
 14  city1_lat             581690 non-null  float64
 15  

,year,quarter,city1,city2,nonstop_miles,passengers,fare,carrier_largest,market_share_largest,fare_largest,carrier_lowfare,market_share_lowfare,fare_lowfare,city-pair,city1_lat,city1_lon,city2_lat,city2_lon
0,2004,2,"Oklahoma City, OK","West Palm Beach/Palm Beach, FL",1197,11,242.51,DL,0.51,235.98,DL,0.51,235.98,"Oklahoma City, OK - West Palm Beach/Palm Beach...",35.472989,-97.517054,26.715364,-80.053294
1,2003,2,"Nashville, TN","West Palm Beach/Palm Beach, FL",758,138,136.00,WN,0.73,126.54,WN,0.73,126.54,"Nashville, TN - West Palm Beach/Palm Beach, FL",36.162277,-86.774298,26.715364,-80.053294
2,2004,4,"Lincoln, NE","San Diego, CA",1259,19,193.88,UA,0.84,194.65,NW,0.13,176.53,"Lincoln, NE - San Diego, CA",40.808886,-96.707775,32.717420,-117.162773
3,2004,2,"Kansas City, MO","Wausau/Mosinee/Stevens Point, WI",459,13,199.57,NW,0.70,201.34,UA,0.13,195.58,"Kansas City, MO - Wausau/Mosinee/Stevens Point...",39.100105,-94.578142,44.522922,-89.574111
4,2003,2,"Allentown/Bethlehem/Easton, PA","Denver, CO",1539,29,272.83,UA,0.32,321.47,DL,0.17,201.91,"Allentown/Bethlehem/Easton, PA - Denver, CO",40.688493,-75.216982,39.739236,-104.984862
5,2009,2,"Miami, FL (Metropolitan Area)","Rochester, NY",1204,203,151.46,FL,0.29,131.05,FL,0.29,131.05,"Miami, FL (Metropolitan Area) - Rochester, NY",25.766368,-80.210268,43.157285,-77.615214
6,2008,4,"Raleigh/Durham, NC","Sarasota/Bradenton, FL",625,24,165.17,DL,0.47,165.67,FL,0.21,154.50,"Raleigh/Durham, NC - Sarasota/Bradenton, FL",35.906111,-78.995782,27.491923,-82.580770
7,2004,3,"Charlotte, NC","Knoxville, TN",177,11,308.14,US,0.94,310.41,DL,0.02,151.90,"Charlotte, NC - Knoxville, TN",35.227209,-80.843083,35.960395,-83.921026
8,2003,3,"Denver, CO","Roanoke, VA",1344,21,237.31,UA,0.38,267.44,NW,0.11,155.75,"Denver, CO - Roanoke, VA",39.739236,-104.984862,37.270973,-79.941431
9,2013,2,"Minneapolis/St. Paul, MN","Savannah, GA",1092,103,222.88,DL,0.70,224.80,UA,0.13,175.85,"Minneapolis/St. Paul, MN - Savannah, GA",44.922705,-93.055492,32.056457,-81.095127


In [13]:
df.carrier_largest.nunique(), df.carrier_lowfare.nunique()

(56, 70)

In [14]:
carriers_df.rename(columns=str.lower, inplace=True)
carriers_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1655 entries, 0 to 1654
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   code     1654 non-null   object
 1   airline  1655 non-null   object
dtypes: object(2)
memory usage: 26.0+ KB


In [15]:
carriers_df.head()

,code,airline
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.


In [16]:
# Two more left joins to add the names of the carriers to the df
df = df.merge(carriers_df, left_on='carrier_largest', right_on='code', how='left')
df.rename(columns={'airline': 'airline_largest'}, inplace=True)
df.drop(['code'], axis=1, inplace=True)

df = df.merge(carriers_df, left_on='carrier_lowfare', right_on='code', how='left')
df.rename(columns={'airline': 'airline_lowfare'}, inplace=True)
df.drop(['code'], axis=1, inplace=True)

In [17]:
df.head(10)

,year,quarter,city1,city2,nonstop_miles,passengers,fare,carrier_largest,market_share_largest,fare_largest,carrier_lowfare,market_share_lowfare,fare_lowfare,city-pair,city1_lat,city1_lon,city2_lat,city2_lon,airline_largest,airline_lowfare
0,2004,2,"Oklahoma City, OK","West Palm Beach/Palm Beach, FL",1197,11,242.51,DL,0.51,235.98,DL,0.51,235.98,"Oklahoma City, OK - West Palm Beach/Palm Beach...",35.472989,-97.517054,26.715364,-80.053294,Delta Air Lines Inc.,Delta Air Lines Inc.
1,2003,2,"Nashville, TN","West Palm Beach/Palm Beach, FL",758,138,136.00,WN,0.73,126.54,WN,0.73,126.54,"Nashville, TN - West Palm Beach/Palm Beach, FL",36.162277,-86.774298,26.715364,-80.053294,Southwest Airlines Co.,Southwest Airlines Co.
2,2004,4,"Lincoln, NE","San Diego, CA",1259,19,193.88,UA,0.84,194.65,NW,0.13,176.53,"Lincoln, NE - San Diego, CA",40.808886,-96.707775,32.717420,-117.162773,United Air Lines Inc.,Northwest Airlines Inc.
3,2004,2,"Kansas City, MO","Wausau/Mosinee/Stevens Point, WI",459,13,199.57,NW,0.70,201.34,UA,0.13,195.58,"Kansas City, MO - Wausau/Mosinee/Stevens Point...",39.100105,-94.578142,44.522922,-89.574111,Northwest Airlines Inc.,United Air Lines Inc.
4,2003,2,"Allentown/Bethlehem/Easton, PA","Denver, CO",1539,29,272.83,UA,0.32,321.47,DL,0.17,201.91,"Allentown/Bethlehem/Easton, PA - Denver, CO",40.688493,-75.216982,39.739236,-104.984862,United Air Lines Inc.,Delta Air Lines Inc.
5,2009,2,"Miami, FL (Metropolitan Area)","Rochester, NY",1204,203,151.46,FL,0.29,131.05,FL,0.29,131.05,"Miami, FL (Metropolitan Area) - Rochester, NY",25.766368,-80.210268,43.157285,-77.615214,AirTran Airways Corporation,AirTran Airways Corporation
6,2008,4,"Raleigh/Durham, NC","Sarasota/Bradenton, FL",625,24,165.17,DL,0.47,165.67,FL,0.21,154.50,"Raleigh/Durham, NC - Sarasota/Bradenton, FL",35.906111,-78.995782,27.491923,-82.580770,Delta Air Lines Inc.,AirTran Airways Corporation
7,2004,3,"Charlotte, NC","Knoxville, TN",177,11,308.14,US,0.94,310.41,DL,0.02,151.90,"Charlotte, NC - Knoxville, TN",35.227209,-80.843083,35.960395,-83.921026,US Airways Inc.,Delta Air Lines Inc.
8,2003,3,"Denver, CO","Roanoke, VA",1344,21,237.31,UA,0.38,267.44,NW,0.11,155.75,"Denver, CO - Roanoke, VA",39.739236,-104.984862,37.270973,-79.941431,United Air Lines Inc.,Northwest Airlines Inc.
9,2013,2,"Minneapolis/St. Paul, MN","Savannah, GA",1092,103,222.88,DL,0.70,224.80,UA,0.13,175.85,"Minneapolis/St. Paul, MN - Savannah, GA",44.922705,-93.055492,32.056457,-81.095127,Delta Air Lines Inc.,United Air Lines Inc.


In [18]:
df.describe()

,year,quarter,nonstop_miles,passengers,fare,market_share_largest,fare_largest,market_share_lowfare,fare_lowfare,city1_lat,city1_lon,city2_lat,city2_lon
count,581690.000000,581690.000000,581690.000000,581690.000000,581690.000000,581690.000000,581690.000000,581690.000000,581690.000000,581690.000000,581690.000000,581690.000000,581690.000000
mean,2009.021142,2.509393,1043.826349,179.120920,232.461192,0.597529,232.999347,0.387582,204.616347,37.456825,-91.378329,36.847805,-94.036642
std,7.671110,1.104600,586.179745,628.420943,70.457895,0.201048,75.422135,0.277607,67.923241,5.293735,13.652916,5.667024,15.941608
min,1996.000000,1.000000,46.000000,10.000000,50.450000,0.000900,50.450000,0.000900,50.000000,24.562557,-124.212652,24.562557,-124.212652
25%,2002.000000,2.000000,589.000000,17.000000,182.000000,0.440000,179.630000,0.160000,155.290000,33.695646,-97.743700,32.717420,-110.974848
50%,2009.000000,3.000000,918.000000,33.000000,229.360000,0.570000,229.640000,0.309000,200.180000,38.046407,-87.624421,37.538509,-90.051764
75%,2016.000000,3.000000,1401.000000,99.000000,278.450000,0.740000,280.970000,0.565200,248.940000,41.875562,-81.690273,40.759620,-81.022833
max,2022.000000,4.000000,2783.000000,25471.000000,754.970000,1.000000,1366.950000,1.000000,1366.950000,48.754401,-67.834909,48.754401,-67.834909


In [19]:
# Uncomment to save processed file to repo
#df.to_csv(r'..\us_airfares_processed.csv')

## Questions to Answer:

* What city-pair was the busiest (top 20)? 
* What was the busiest airport each quarter?
* Pre-Covid vs Post-Covid -- Average ticket price? Passengers in top markets?

### 

### Top Routes Evaluated by Passenger Volume

In [20]:
pax_vol = df.groupby('city-pair').agg({'passengers':'mean'}).sort_values(by='passengers', ascending=False)

display(pax_vol)

fig = px.histogram(pax_vol, x='passengers')
fig.show()

,passengers
city-pair,
"Los Angeles, CA (Metropolitan Area) - San Francisco, CA (Metropolitan Area)",18506.261682
"Miami, FL (Metropolitan Area) - New York City, NY (Metropolitan Area)",13559.495327
"Los Angeles, CA (Metropolitan Area) - New York City, NY (Metropolitan Area)",9957.532710
"New York City, NY (Metropolitan Area) - Orlando, FL",8780.897196
"Chicago, IL - New York City, NY (Metropolitan Area)",8719.943925
...,...
"Columbus, GA - Minneapolis/St. Paul, MN",10.000000
"Columbus, GA - El Paso, TX",10.000000
"Columbia, SC - Valparaiso, FL",10.000000


### Miles vs Fares 

In [21]:
df['fare_per_mile'] = df.fare / df.nonstop_miles
# range_selection = st.slider('Select the time range for the plot:',
#                             df.year.min(),
#                             df.year.max(),
#                             (1999, 2013))
# flight_year_df = df[df['year'].between(range_selection[0],
#                                        range_selection[1], 
#                                        inclusive='both'
#                                       )]

flight_year_df = df[df.year == 2003]

fig_a = px.scatter(flight_year_df, 
                   x='nonstop_miles', 
                   y='fare',
                   trendline='ols', 
                   color='airline_largest', 
                   color_continuous_scale='speed')
fig_a.show()

### Geoplot Top Routes by Revenue

In [22]:
df['revenue'] = df['passengers'] * df['fare']

revenue_year = 2017
size_select = 100

revenue_df = df[df['year']==revenue_year].groupby('city-pair').agg({'revenue':'sum',
                                                                    'passengers':'sum',
                                                                    'airline_largest':lambda x:x.value_counts().index[0],
                                                                    'city1':'first',
                                                                    'city2':'first',
                                                                    'city1_lon':'first',
                                                                    'city2_lon':'first',
                                                                    'city1_lat':'first',
                                                                    'city2_lat':'first'})\
                                            .sort_values(by='revenue', ascending=False).head(size_select)



In [23]:
revenue_df

,revenue,passengers,airline_largest,city1,city2,city1_lon,city2_lon,city1_lat,city2_lat
city-pair,,,,,,,,,
"Los Angeles, CA (Metropolitan Area) - New York City, NY (Metropolitan Area)",19263013.30,51042,United Air Lines Inc.,"Los Angeles, CA (Metropolitan Area)","New York City, NY (Metropolitan Area)",-118.476863,-74.002738,33.995431,40.716649
"New York City, NY (Metropolitan Area) - San Francisco, CA (Metropolitan Area)",14309167.12,40227,United Air Lines Inc.,"New York City, NY (Metropolitan Area)","San Francisco, CA (Metropolitan Area)",-74.002738,-122.419906,40.716649,37.779026
"Los Angeles, CA (Metropolitan Area) - San Francisco, CA (Metropolitan Area)",11348475.02,87549,Southwest Airlines Co.,"Los Angeles, CA (Metropolitan Area)","San Francisco, CA (Metropolitan Area)",-118.476863,-122.419906,33.995431,37.779026
"Miami, FL (Metropolitan Area) - New York City, NY (Metropolitan Area)",10337203.78,63334,Delta Air Lines Inc.,"Miami, FL (Metropolitan Area)","New York City, NY (Metropolitan Area)",-80.210268,-74.002738,25.766368,40.716649
"Chicago, IL - New York City, NY (Metropolitan Area)",7954732.97,46361,United Air Lines Inc.,"Chicago, IL","New York City, NY (Metropolitan Area)",-87.624421,-74.002738,41.875562,40.716649
...,...,...,...,...,...,...,...,...,...
"Boston, MA (Metropolitan Area) - Dallas/Fort Worth, TX",1782785.47,8292,American Airlines Inc.,"Boston, MA (Metropolitan Area)","Dallas/Fort Worth, TX",-71.060511,-97.239737,42.355433,32.737906
"Dallas/Fort Worth, TX - Miami, FL (Metropolitan Area)",1778515.48,8280,American Airlines Inc.,"Dallas/Fort Worth, TX","Miami, FL (Metropolitan Area)",-97.239737,-80.210268,32.737906,25.766368
"Denver, CO - Houston, TX",1762383.71,8874,Southwest Airlines Co.,"Denver, CO","Houston, TX",-104.984862,-95.367697,39.739236,29.758938


In [24]:
revenue_df[['passengers', 'revenue']]

,passengers,revenue
city-pair,,
"Los Angeles, CA (Metropolitan Area) - New York City, NY (Metropolitan Area)",51042,19263013.30
"New York City, NY (Metropolitan Area) - San Francisco, CA (Metropolitan Area)",40227,14309167.12
"Los Angeles, CA (Metropolitan Area) - San Francisco, CA (Metropolitan Area)",87549,11348475.02
"Miami, FL (Metropolitan Area) - New York City, NY (Metropolitan Area)",63334,10337203.78
"Chicago, IL - New York City, NY (Metropolitan Area)",46361,7954732.97
...,...,...
"Boston, MA (Metropolitan Area) - Dallas/Fort Worth, TX",8292,1782785.47
"Dallas/Fort Worth, TX - Miami, FL (Metropolitan Area)",8280,1778515.48
"Denver, CO - Houston, TX",8874,1762383.71


In [25]:
revenue_df.index.nunique()

100

In [26]:
plot_airports_df = revenue_df[['city1', 'city1_lon', 'city1_lat']]
plt_apts2 = revenue_df[['city2', 'city2_lon', 'city2_lat']]
plt_apts2.columns = plot_airports_df.columns
plot_airports_df = pd.concat([plot_airports_df, plt_apts2], axis=0)

plot_airports_df.head(10)

,city1,city1_lon,city1_lat
city-pair,,,
"Los Angeles, CA (Metropolitan Area) - New York City, NY (Metropolitan Area)","Los Angeles, CA (Metropolitan Area)",-118.476863,33.995431
"New York City, NY (Metropolitan Area) - San Francisco, CA (Metropolitan Area)","New York City, NY (Metropolitan Area)",-74.002738,40.716649
"Los Angeles, CA (Metropolitan Area) - San Francisco, CA (Metropolitan Area)","Los Angeles, CA (Metropolitan Area)",-118.476863,33.995431
"Miami, FL (Metropolitan Area) - New York City, NY (Metropolitan Area)","Miami, FL (Metropolitan Area)",-80.210268,25.766368
"Chicago, IL - New York City, NY (Metropolitan Area)","Chicago, IL",-87.624421,41.875562
"New York City, NY (Metropolitan Area) - Orlando, FL","New York City, NY (Metropolitan Area)",-74.002738,40.716649
"Los Angeles, CA (Metropolitan Area) - Washington, DC (Metropolitan Area)","Los Angeles, CA (Metropolitan Area)",-118.476863,33.995431
"Atlanta, GA (Metropolitan Area) - New York City, NY (Metropolitan Area)","Atlanta, GA (Metropolitan Area)",-84.391906,33.758891
"San Francisco, CA (Metropolitan Area) - Washington, DC (Metropolitan Area)","San Francisco, CA (Metropolitan Area)",-122.419906,37.779026


In [27]:
fig_rev = go.Figure()

for i in range(len(revenue_df)):
    fig_rev.add_trace(
        go.Scattergeo(
            locationmode='USA-states',
            lon=[revenue_df['city1_lon'][i], revenue_df['city2_lon'][i]],
            lat=[revenue_df['city1_lat'][i], revenue_df['city2_lat'][i]],
            mode = 'lines',
            line = dict(width = 1,color = 'red'),
            opacity = float(i) / float(size_select)
        )
    )

In [28]:
fig_rev.add_trace(go.Scattergeo(
    #locationmode = "USA-states",
    lon = plot_airports_df['city1_lon'],
    lat = plot_airports_df['city1_lat'],
    hoverinfo= 'text',
    text = plot_airports_df['city1'],
    mode='markers',
    marker=dict(
            size = 5,
            color = 'rgb(255, 0, 0)',
            line = dict(
            width = 20,
            color = 'rgba(68, 68, 68, 0)'
            ),
    
    )))

In [29]:
fig_rev.update_layout(
    title_text = 'Biggest Markets in the USA by Revenue',
    title_x=0.4,
    title_y=0.9,
    showlegend = False,
    autosize = True,
    width=1200, height=720,    
    geo = dict(
        scope = 'north america',
        projection_type = 'azimuthal equal area',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)'           
    ),
)
fig_rev.update_geos(fitbounds='locations')

In [30]:
fig_rev.show()

In [31]:
df

,year,quarter,city1,city2,nonstop_miles,passengers,fare,carrier_largest,market_share_largest,fare_largest,...,fare_lowfare,city-pair,city1_lat,city1_lon,city2_lat,city2_lon,airline_largest,airline_lowfare,fare_per_mile,revenue
0,2004,2,"Oklahoma City, OK","West Palm Beach/Palm Beach, FL",1197,11,242.51,DL,0.5100,235.98,...,235.98,"Oklahoma City, OK - West Palm Beach/Palm Beach...",35.472989,-97.517054,26.715364,-80.053294,Delta Air Lines Inc.,Delta Air Lines Inc.,0.202598,2667.61
1,2003,2,"Nashville, TN","West Palm Beach/Palm Beach, FL",758,138,136.00,WN,0.7300,126.54,...,126.54,"Nashville, TN - West Palm Beach/Palm Beach, FL",36.162277,-86.774298,26.715364,-80.053294,Southwest Airlines Co.,Southwest Airlines Co.,0.179420,18768.00
2,2004,4,"Lincoln, NE","San Diego, CA",1259,19,193.88,UA,0.8400,194.65,...,176.53,"Lincoln, NE - San Diego, CA",40.808886,-96.707775,32.717420,-117.162773,United Air Lines Inc.,Northwest Airlines Inc.,0.153995,3683.72
3,2004,2,"Kansas City, MO","Wausau/Mosinee/Stevens Point, WI",459,13,199.57,NW,0.7000,201.34,...,195.58,"Kansas City, MO - Wausau/Mosinee/Stevens Point...",39.100105,-94.578142,44.522922,-89.574111,Northwest Airlines Inc.,United Air Lines Inc.,0.434793,2594.41
4,2003,2,"Allentown/Bethlehem/Easton, PA","Denver, CO",1539,29,272.83,UA,0.3200,321.47,...,201.91,"Allentown/Bethlehem/Easton, PA - Denver, CO",40.688493,-75.216982,39.739236,-104.984862,United Air Lines Inc.,Delta Air Lines Inc.,0.177277,7912.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581685,2022,3,"Knoxville, TN","Tulsa, OK",666,16,381.33,DL,0.5139,405.27,...,367.80,"Knoxville, TN - Tulsa, OK",35.960395,-83.921026,36.156312,-95.992752,Delta Air Lines Inc.,American Airlines Inc.,0.572568,6101.28
581686,2022,3,"Knoxville, TN","Sanford, FL",511,259,87.53,G4,1.0000,87.53,...,87.53,"Knoxville, TN - Sanford, FL",35.960395,-83.921026,28.811735,-81.268022,Allegiant Air,Allegiant Air,0.171292,22670.27
581687,2022,3,"Knoxville, TN","Sarasota/Bradenton, FL",586,17,254.13,DL,0.5714,285.29,...,79.44,"Knoxville, TN - Sarasota/Bradenton, FL",35.960395,-83.921026,27.491923,-82.580770,Delta Air Lines Inc.,Allegiant Air,0.433669,4320.21
581688,2022,3,"Knoxville, TN","Syracuse, NY",656,16,307.66,AA,0.5103,294.85,...,294.85,"Knoxville, TN - Syracuse, NY",35.960395,-83.921026,43.048122,-76.147424,American Airlines Inc.,American Airlines Inc.,0.468994,4922.56
